In [ ]:
!gdown --id 1o7B8XJ33OjCo8paZytvvVsxncm9wvVke


Downloading...
From: https://drive.google.com/uc?id=1o7B8XJ33OjCo8paZytvvVsxncm9wvVke
To: /content/BigData_Homework1_Data.zip
377MB [00:03, 117MB/s]


In [ ]:
!unzip BigData_Homework1_Data.zip

Archive:  BigData_Homework1_Data.zip
   creating: Data/
   creating: Data/question 2_Association rules' application/
  inflating: Data/question 2_Association rules' application/browsing.txt  
   creating: Data/question 3_LSH/
   creating: Data/question 3_LSH/data/
  inflating: Data/question 3_LSH/data/patches.csv  
  inflating: Data/question 3_LSH/lsh.py  
   creating: Data/question 4_Data streams/
  inflating: Data/question 4_Data streams/counts.txt  
  inflating: Data/question 4_Data streams/counts_tiny.txt  
  inflating: Data/question 4_Data streams/hash_params.txt  
  inflating: Data/question 4_Data streams/words_stream.txt  
  inflating: Data/question 4_Data streams/words_stream_tiny.txt  


In [ ]:
import csv
import numpy as np
import pandas as pd
from itertools import combinations

In [ ]:
def create_candidate_1(X):
    """
    create the 1-item candidate,
    it's basically creating a frozenset for each unique item
    and storing them in a list
    """
    c1 = []
    for transaction in X:
        for t in transaction:
            t = frozenset([t])
            if t not in c1:
                c1.append(t)
    return c1

In [ ]:
def create_candidate_k(freq_item, k):
    """create the list of k-item candidate"""
    ck = []

    # for generating candidate of size two (2-itemset)
    if k == 0:
        for f1, f2 in combinations(freq_item, 2):
            item = f1 | f2  # union of two sets
            ck.append(item)
    else:
        for f1, f2 in combinations(freq_item, 2):
            # if the two (k+1)-item sets has
            # k common elements then they will be
            # unioned to be the (k+2)-item candidate
            intersection = f1 & f2
            if len(intersection) == k:
                item = f1 | f2
                if item not in ck:
                    ck.append(item)
    return ck

In [ ]:
def create_freq_item(X, ck, min_support):
    """
    filters the candidate with the specified
    minimum support
    """
    # loop through the transaction and compute
    # the count for each candidate (item)
    item_count = {}
    for transaction in X:
        for item in ck:
            if item.issubset(transaction):
                if item not in item_count:
                    item_count[item] = 1
                else:
                    item_count[item] += 1

    n_row = X.shape[0]
    freq_item = []
    item_support = {}

    # if the support of an item is greater than the
    # min_support, then it is considered as frequent
    for item in item_count:
        support = item_count[item] / n_row
        if support >= min_support:
            freq_item.append(item)

        item_support[item] = item_count[item]

    return freq_item, item_support

In [ ]:
def apriori(X, min_support):
    """
    pass in the transaction data and the minimum support
    threshold to obtain the frequent itemset. Also
    store the support for each itemset, they will
    be used in the rule generation step
    """

    # the candidate sets for the 1-item is different,
    # create them independently from others
    c1 = create_candidate_1(X)
    freq_item, item_support_dict = create_freq_item(X, c1, min_support)
    freq_items = [freq_item]

    k = 0
    while len(freq_items[k]) > 0:
        freq_item = freq_items[k]
        ck = create_candidate_k(freq_item, k)
        freq_item, item_support = create_freq_item(X, ck, min_support)
        if(k==0):
            with open('output.txt','w') as f:
                for k, v in item_support.items():
                    # print(str(list(k)) + str(v))
                    if(v>=100):
                      f.write("itemset: "+(str(list(k))) + " count: "+(str(v))+"\n")

            break
        freq_items.append(freq_item)
        item_support_dict.update(item_support)
        k += 1

    return freq_items, item_support_dict

In [ ]:
trans = []
with open('/content/Data/question2/browsing.txt') as f:
    contents = f.readlines()
    for i in contents:
        trans.append(i.split())
trans[0]

['FRO11987', 'ELE17451', 'ELE89019', 'SNA90258', 'GRO99222']

In [ ]:
c1 = create_candidate_1(trans)


In [ ]:
trans=np.array(trans)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [ ]:
freq_item, item_support_dict = create_freq_item(trans, c1, 0.003)

In [ ]:
item_support_dict

In [ ]:
ck = create_candidate_k(freq_item, 0)

In [ ]:
 item_count = {}
 for transaction in trans:
      for item in ck:
          if item.issubset(transaction):
              if item not in item_count:
                  item_count[item] = 1
              else:
                  item_count[item] += 1

In [ ]:
len(item_count)

162037

In [ ]:
conf_list=[]
for k, v in item_count.items():
      X_and_Y = list(k)
      support_X = item_support_dict.get(frozenset({X_and_Y[0]}))
      support_Y = item_support_dict.get(frozenset({X_and_Y[1]}))
      conf_list.append([X_and_Y,float(v/support_X)])
      Y_and_X = X_and_Y[::-1]
      conf_list.append([Y_and_X,float(v/support_Y)])
conf_list[:5]

[[['FRO11987', 'ELE17451'], 0.17307692307692307],
 [['ELE17451', 'FRO11987'], 0.00464516129032258],
 [['SNA90258', 'FRO11987'], 0.0036363636363636364],
 [['FRO11987', 'SNA90258'], 0.019230769230769232],
 [['FRO11987', 'GRO99222'], 0.038461538461538464]]

In [ ]:
sorted_list = sorted(conf_list, key=lambda x: x[1],reverse=True)
(sorted_list[:5])

[[['DAI93865', 'FRO40251'], 1.0],
 [['GRO85051', 'FRO40251'], 0.999176276771005],
 [['GRO38636', 'FRO40251'], 0.9906542056074766],
 [['ELE12951', 'FRO40251'], 0.9905660377358491],
 [['DAI88079', 'FRO40251'], 0.9867256637168141]]

In [ ]:
c2 = ck
#now we go to part 2:
freq_item2, item_support2 = create_freq_item(trans, c2, 0.003)

In [ ]:
c3 = create_candidate_k(freq_item2, 1)

In [ ]:
triple_item_count = {}
for transaction in trans:
    for item in c3:
        if item.issubset(transaction):
            if item not in triple_item_count:
                triple_item_count[item] = 1
            else:
                triple_item_count[item] += 1

In [ ]:
triple_conf_list = []
for k, v in triple_item_count.items():
    X_and_Y_and_Z = list(k)
    support_X_Y = item_count.get(frozenset({X_and_Y_and_Z[0],X_and_Y_and_Z[1]}))
    support_Y_Z = item_count.get(frozenset({X_and_Y_and_Z[1],X_and_Y_and_Z[2]}))
    support_X_Z = item_count.get(frozenset({X_and_Y_and_Z[0],X_and_Y_and_Z[2]}))
    triple_conf_list.append([X_and_Y_and_Z, float(v / support_X_Y)])
    Z_and_Y_and_X = X_and_Y_and_Z[::-1]
    triple_conf_list.append([Z_and_Y_and_X, float(v / support_Y_Z)])
    Z_and_Y_and_X[1], Z_and_Y_and_X[2] = Z_and_Y_and_X[2], Z_and_Y_and_X[1]
    triple_conf_list.append([Z_and_Y_and_X, float(v / support_X_Z)])
sorted_list = sorted(triple_conf_list, key=lambda x: x[1], reverse=True)
(sorted_list[:5])


[[['GRO56989', 'DAI50921', 'DAI62779'], 1.0],
 [['GRO38636', 'FRO40251', 'ELE17451'], 1.0],
 [['GRO38636', 'SNA93860', 'FRO40251'], 1.0],
 [['FRO31317', 'FRO40251', 'GRO38636'], 1.0],
 [['DAI62779', 'GRO38636', 'FRO40251'], 1.0]]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
